In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest import MCMCSampler

In [4]:
# Himmelblau
def loglike(z):
    z1 = z[:, 0]
    z2 = z[:, 1]
    return - (z1**2 + z2 - 11.)**2 - (z1 + z2**2 - 7.)**2
def transform(x):
    return 5. * x
dims = 2

In [5]:
# Rosenbrock
def loglike(z):
    return np.array([-sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1 - x[:-1]) ** 2.0) for x in z])
def transform(x):
    return 5. * x
dims = 2

In [6]:
sampler = MCMCSampler(dims, loglike, transform=transform, hidden_dim=128, num_layers=1, num_blocks=5, scale='')

Creating directory for new run logs/test/run11
[nnest.trainer] [INFO] SingleSpeed(
  (net): FlowSequential(
    (0): CouplingLayer(
      (scale_net): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): Tanh()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Tanh()
        (4): Linear(in_features=128, out_features=2, bias=True)
      )
      (translate_net): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): ReLU()
        (4): Linear(in_features=128, out_features=2, bias=True)
      )
    )
    (1): CouplingLayer(
      (scale_net): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): Tanh()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Tanh()
        (4): Linear(in_features=128, out_features=2, bias=True)
      )
      (translat

In [ ]:
sampler.run()

[nnest.sampler] [INFO] Alpha [0.7071]
[nnest.sampler] [INFO] Getting initial samples with emcee
[nnest.sampler] [INFO] Mean acceptance fraction: [0.419]
Removed no burn in
[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training noise [0.0005]
[nnest.trainer] [INFO] Epoch [1] train loss [0.0234] validation loss [0.0233]
[nnest.trainer] [INFO] Epoch [50] train loss [0.0060] validation loss [0.0057]
